In [1]:
import IPython.display as ipd
import os
import torch
import numpy as np
import utils

from models import SynthesizerTrn

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

hps = utils.get_hparams_from_file("./configs/vctk_hubert_cn.json")
net_g = SynthesizerTrn(
    hps.data.filter_length // 2 + 1,
    # 512,
    hps.train.segment_size // hps.data.hop_length,
    # hps.train.segment_size // 320,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("logs/cn_no_f0_1123/G_250000.pth", net_g, None)

INFO:root:Loaded checkpoint 'logs/cn_no_f0_1123/G_250000.pth' (iteration 3)


In [3]:
sid = np.load('/data/public/dataset/aishell3_16k/sid_ecapa_tdnn/SSB15670096.npy')
tid = np.load('/data/public/dataset/aishell3_16k/sid_ecapa_tdnn/SSB15670096.npy')

sunit = np.load('/data/public/dataset/aishell3_16k/hubert_soft/SSB15670096.npy')
tunit = np.load('/data/public/dataset/aishell3_16k/hubert_soft/SSB15670096.npy')

with torch.no_grad():
    tid = torch.FloatTensor(tid).cuda()
    sid = torch.FloatTensor(sid).cuda()
    sunit = torch.FloatTensor(sunit).cuda()
    tunit = torch.FloatTensor(tunit).cuda()

    tid.unsqueeze_(0)
    sid.unsqueeze_(0)
    sunit.unsqueeze_(0)
    tunit.unsqueeze_(0)


    sunit_length = torch.LongTensor([sunit.size(1)]).cuda()
    tunit_length = torch.LongTensor([tunit.size(1)]).cuda()

In [7]:
print("s to t")
audio1 = net_g.infer(sunit.transpose(1,2), sunit_length, sid=tid, noise_scale=.667, noise_scale_w=0.8, length_scale=1.5)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))

s to t
